# pandapower WLS State Estimation
This is an introduction into the usage of the pandapower state estimation module. It shows how to 
create measurements for a pandapower network and how to perform a state estimation with the weighted least squares (WLS) algorithm.

## Example Network

We will be using the reference network from the book "Power System State Estimation" by Ali Abur and Antonio Gómez Expósito. 
It contains 3 buses with connecting lines between buses 1-2, 1-3 and 2-3. 8 measurements of different types enable WLS state estimation.

We first create this network in pandapower.

In [1]:
import pandapower as pp

net = pp.create_empty_network()

b1 = pp.create_bus(net, name="bus 1", vn_kv=1., index=1)
b2 = pp.create_bus(net, name="bus 2", vn_kv=1., index=2)
b3 = pp.create_bus(net, name="bus 3", vn_kv=1., index=3)

pp.create_ext_grid(net, 1)  # set the slack bus to bus 1

l1 = pp.create_line_from_parameters(net, 1, 2, 1, r_ohm_per_km=.01, x_ohm_per_km=.03, c_nf_per_km=0., imax_ka=1)
l2 = pp.create_line_from_parameters(net, 1, 3, 1, r_ohm_per_km=.02, x_ohm_per_km=.05, c_nf_per_km=0., imax_ka=1)
l3 = pp.create_line_from_parameters(net, 2, 3, 1, r_ohm_per_km=.03, x_ohm_per_km=.08, c_nf_per_km=0., imax_ka=1)

net

SyntaxError: invalid syntax (create.py, line 15)

Now we can add our measurements, which are valid for one point in time.

We add two voltage magnitude measurements on buses 1 / 2 with voltage magnitude of 1.006 pu / 0.968 pu and a standard deviation of 0.004 pu each:

## Adding Measurements

Measurements are defined via the pandapower *create_measurement* function.    
The physical properties which can be measured are set with the *type* argument and can be one of the following: "p" for active power, "q" for reactive power, "v" for voltage and "i" for electrical current.   
The element is set with the *element_type* argument, it can be either "bus", "line" or "transformer".    
Power is measured in kW / kVar, voltage in per unit and current in A. Bus power injections are positive if power is generated at the bus and negative if it is consumed.

In [ ]:
pp.create_measurement(net, "v", "bus", 1.006, .004, b1)        # V at bus 1
pp.create_measurement(net, "v", "bus", 0.968, .004, b2)        # V at bus 2
net.measurement

We add bus injection measurements on bus 2 with P=-501 kW and Q=-286kVar and standard deviations of 10kVA: 

In [ ]:
pp.create_measurement(net, "p", "bus", -501, 10, b2)         # P at bus 2
pp.create_measurement(net, "q", "bus", -286, 10, b2)         # Q at bus 2
net.measurement

Finally, we add line measurements for lines 0 and 1, both placed at the side of bus 1. The bus parameter defines the bus at which the line measurement is positioned, the line argument is the index of the line.

In [ ]:
pp.create_measurement(net, "p", "line", 888, 8, bus=b1, element=l1)    # Pline (bus 1 -> bus 2) at bus 1
pp.create_measurement(net, "p", "line", 1173, 8, bus=b1, element=l2)   # Pline (bus 1 -> bus 3) at bus 1
pp.create_measurement(net, "q", "line", 568, 8, bus=b1, element=l1)    # Qline (bus 1 -> bus 2) at bus 1
pp.create_measurement(net, "q", "line", 663, 8, bus=b1, element=l2)    # Qline (bus 1 -> bus 3) at bus 1
net.measurement

## Performing the State Estimation

The measurements are now set. We have to initialize the starting voltage magnitude and voltage angles for the state estimator. In continous operation, this can be the result of the last state estimation. In our case, we set flat start conditions: 1.0 p.u. for voltage magnitude, 0.0 degree for voltage angles. This is easily done with the parameter "init", which we define as "flat".

And now run the state estimation. Afterwards, the result will be stored in the table res_bus_est.

In [ ]:
from pandapower.estimation import estimate

success = estimate(net, init='flat')
print(success)

## Handling of Bad Data

The state estimation class allows additionally the removal of bad data, especially single or non-interacting false measurements. For detecting bad data the Chi-square distribution is used to identify the presence of them. Afterwards follows the largest normalized residual test that identifys the actual measurements which will be removed at the end.

To test this function we will add a single false measurement (active power of line 1 at bus 3) to the network:

In [ ]:
pp.create_measurement(net, "p", "line", 1000, 8, bus=b3, element=l1)    # Pline (bus 1 -> bus 2) at bus 3
net.measurement

The next step is the call of the "rn_max"-function that handles the removal of the added false measurement, and returns the voltages (with their magnitudes and angles as numpy arrays) of the "false-free" state estimation as well as its success.

In [ ]:
from pandapower.estimation import state_estimation
import numpy as np

# initial state values ('flat' start)
v_start = np.array([1.0, 1.0, 1.0])
delta_start = np.array([0., 0., 0.])
    
# 1. Initialize state estimation class
wls = state_estimation(net = net)

# 2. Perform rn_max test (includes Chi-square analysis)
v_est_rn_max, delta_est_rn_max, success_rn_max = wls.perform_rn_max_test(v_start, delta_start)

The results will be the same like the ones from the following section.

## Working with Results

We can show the voltage magnitude and angles directly:

In [ ]:
net.res_bus_est.vm_pu

In [ ]:
net.res_bus_est.va_degree

The results match exactly with the results from the book: Voltages 0.9996, 0.9742, 0.9439; Voltage angles 0.0, -1.2475, -2.7457). Nice!    
Let's look at the bus power injections, which are available in res_bus_est as well

In [ ]:
net.res_bus_est.p_kw

In [ ]:
net.res_bus_est.q_kvar

We can also compare the resulting line power flows with the measurements.

In [ ]:
net.res_line_est.p_from_kw

In [ ]:
net.res_line_est.q_from_kvar

Again, this values do match the estimated values from our reference book.
This concludes the small tutorial how to perform state estimation with a pandapower network.